# Compute Human-Machine Alignment

This notebook computes the alignment between moves made by humans and the machine. To do so, we load the full trajectory of human moves, and predict for each move the best move according to the machine. The machine does so taking into account the trajectory of human play. We encode with one, if human and machine move matches.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# add dqn sub folder to python path
import sys
sys.path.append('dqn')

In [ ]:
import pandas as pd
import os

import yaml
import json
import os

import torch as th
from dqn.environment_vect import Reward_Network
from dqn.dqn_agent import Agent
from dqn.config_type import Config

In [ ]:
experiment_type = 'reward-network-iii-exp-v2'

moves_df = pd.read_csv(os.path.join('../data/exp_processed/moves.csv'))

networks_file = '../data/networks_solutions_models/solution_exp__valid_networks.json'

# Load the networks
with open(networks_file) as f:
    networks_exp = json.load(f)

config_file = '../algorithm/params/seed_0.yml'
with open(config_file) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
config = Config(**config)


In [ ]:
moves_df['session_trial_id'] = moves_df['session_id'].astype(str) + '_' + moves_df['trial_id'].astype(str)

session_trial_id = moves_df['session_trial_id'].unique()
session_trial_idx = {session_trial_id[i]: i for i in range(len(session_trial_id))}

In [ ]:
human_moves = th.full((len(session_trial_id), 10), -1, dtype=th.int64)
network_ids = []

for _, row in moves_df.iterrows():
    idx = session_trial_idx[row['session_trial_id']]
    human_moves[idx, row['move_idx']] = row['target_num']
    if row['move_idx'] == 0:
        network_ids.append(row['network_id'])

In [ ]:
networks_keyed = {network['network_id']: network for network in networks_exp}
networks_reorganized = [networks_keyed[network_id] for network_id in network_ids]

In [ ]:
DEVICE = th.device("cuda" if th.cuda.is_available() else "cpu")

env = Reward_Network(networks_reorganized, network_batch=None, config=config, device=DEVICE)

AI_agent = Agent(
    observation_shape=env.observation_shape,
    config=config,
    action_dim=env.action_space_idx.shape,
    save_dir=None,
    device=DEVICE,
)
path = os.path.join('..',config.save_dir, f"{config.name}_{config.seed}.pt")
AI_agent.load_model(path)

# solve all networks at once in one go ("1 episode")
ai_moves = AI_agent.solve_loop(
    episode=0,
    n_rounds=config.n_rounds,
    train_mode=False,
    exp_mode=True,
    env=env,
    logger=None,
    exec_actions=human_moves,
)

In [ ]:
moves_df['ai_target_num'] = None

for idx, row in moves_df.iterrows():
    session_idx = session_trial_idx[row['session_trial_id']]
    move_idx = row['move_idx']
    moves_df.at[idx, 'ai_target_num'] = ai_moves[session_idx, move_idx]

In [ ]:
moves_df['human_machine_match'] = moves_df['ai_target_num'] == moves_df['target_num']

In [ ]:
moves_df.to_csv(os.path.join('../data/exp_processed/moves_w_alignment.csv'), index=False)

In [ ]:
import seaborn as sns

df = moves_df[moves_df['trial_type'] == 'demonstration']

sns.lineplot(data=moves_df, x='generation', hue='condition', style='ai_player', y='human_machine_match', markers=True)
